In [ ]:
!git clone --depth 1 https://github.com/spMohanty/PlantVillage-Dataset.git

In [ ]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, average_precision_score
import matplotlib.pyplot as plt
import torch.nn.functional as F
import random
!pip install ultralytics
from ultralytics import YOLO  # For YOLOv11

In [ ]:
# Define paths
dataset_path = 'PlantVillage-Dataset/raw/color'

# Get list of all images and their labels
images = []
labels = []
class_names = os.listdir(dataset_path)
class_to_idx = {class_name: idx for idx, class_name in enumerate(class_names)}

# Fix the total number of samples per class to 500
fixed_samples_per_class = 500

for class_name in class_names:
    class_path = os.path.join(dataset_path, class_name)
    img_paths = [os.path.join(class_path, img_name) for img_name in os.listdir(class_path)]

    # Randomly sample 500 images per class (or take all if fewer)
    img_paths = random.sample(img_paths, min(len(img_paths), fixed_samples_per_class))

    images.extend(img_paths)
    labels.extend([class_to_idx[class_name]] * len(img_paths))

# Split the dataset into training (60%), validation (20%), and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2






class PlantVillageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = cv2.imread(self.image_paths[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
        image = cv2.resize(image, (224, 224))  # Resize to 224x224
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Create datasets
train_dataset = PlantVillageDataset(X_train, y_train, transform=transform)
val_dataset = PlantVillageDataset(X_val, y_val, transform=transform)
test_dataset = PlantVillageDataset(X_test, y_test, transform=transform)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Train Samples: {len(train_dataset)}, Val Samples: {len(val_dataset)}, Test Samples: {len(test_dataset)}")



In [ ]:
def train_model(model_name, train_loader, val_loader, test_loader, num_classes, num_epochs=5):
    # Load YOLO model (smallest version)
    if model_name == 'YOLOv11':
        model = YOLO('yolov11n.pt')  # Smallest version of YOLOv11
    else:
        raise ValueError("Model not supported")

    # Move model to GPU if available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    train_losses, val_losses = [], []
    train_accs, val_accs = [], []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_acc = 100.0 * correct / total
        train_losses.append(train_loss)
        train_accs.append(train_acc)

        # Validation
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        val_loss = val_loss / len(val_loader)
        val_acc = 100.0 * correct / total
        val_losses.append(val_loss)
        val_accs.append(val_acc)

        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')

    # Plot training and validation accuracy
    plt.plot(train_accs, label='Training Accuracy')
    plt.plot(val_accs, label='Validation Accuracy')
    plt.title(f'{model_name} Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

    # Evaluate on test set
    model.eval()
    y_true, y_scores = [], []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            probs = F.softmax(outputs, dim=1).cpu().numpy()
            y_true.extend(labels.cpu().numpy())
            y_scores.extend(probs)

    # Plot PR curves for top-N classes
    y_true = np.array(y_true)
    y_scores = np.array(y_scores)
    top_n = 5
    ap_scores = []

    for class_idx in range(num_classes):
        average_precision = average_precision_score(y_true == class_idx, y_scores[:, class_idx])
        ap_scores.append((class_idx, average_precision))

    top_classes = sorted(ap_scores, key=lambda x: x[1], reverse=True)[:top_n]

    plt.figure(figsize=(10, 8))
    for class_idx, average_precision in top_classes:
        precision, recall, _ = precision_recall_curve(y_true == class_idx, y_scores[:, class_idx])
        plt.plot(recall, precision, label=f'Class {class_idx} (AP={average_precision:.2f})')

    plt.title(f'{model_name} Precision-Recall Curve (Top-{top_n} Classes)', fontsize=16)
    plt.xlabel('Recall', fontsize=14)
    plt.ylabel('Precision', fontsize=14)
    plt.legend(loc='best', fontsize=12)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

    return model


In [ ]:
model = 'YOLOv11'
result = train_model(model, train_loader, val_loader, test_loader, num_classes=38, num_epochs=5)